In [3]:
import tensorflow as tf
import json
import base64
import requests


def prepare_example(review_text):
    # Define the features of your data
    feature = {
        "Review": tf.train.Feature(bytes_list=tf.train.BytesList(value=[review_text.encode()]))
    }

    # Create an Example
    example = tf.train.Example(features=tf.train.Features(feature=feature))

    # Serialize the Example to a string
    example_str = example.SerializeToString()

    # Prepare the JSON data
    json_data = {
        "signature_name": "serving_default",
        "instances": [{"b64": base64.b64encode(example_str).decode()}]
    }

    return json.dumps(json_data)

json_data = prepare_example("It's so bad! I hate it!")


endpoint = "https://orchestrate-restaurant-review-production.up.railway.app/v1/models/review-model:predict"
response = requests.post(endpoint, data=json_data)

prediction = response.json().get("predictions")
if prediction:
    prediction_value = prediction[0][0]
    result = "Good" if prediction_value > 0.6 else "Bad"
else:
    result = "Error: No predictions found in response."

print(result)

Bad
